In [25]:
import json
import plotly
import pandas as pd
import matplotlib

import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from flask import Flask
from flask import render_template, request, jsonify
from plotly.graph_objs import Bar
from sklearn.externals import joblib
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [26]:
import pandas as pd


In [27]:
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('Messages', engine)


In [28]:
df.head()


,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [50]:
df.shape[0]

26215

In [78]:
df2 =  df[df['genre']=='direct'].iloc[:,4:].sum()

In [84]:
df2

request                   3696
offer                       46
aid_related               4338
medical_help               592
medical_products           471
search_and_rescue          216
security                   131
military                    46
child_alone                  0
water                      836
food                      1807
shelter                   1152
clothing                   247
money                      148
missing_people              86
refugees                   174
death                      254
other_aid                 1575
infrastructure_related     327
transport                  210
buildings                  391
electricity                 81
tools                       33
hospitals                   54
shops                       33
aid_centers                 78
other_infrastructure       186
weather_related           1521
floods                     304
storm                      315
fire                        41
earthquake                 796
cold    

In [77]:
df2.index

Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [61]:
(df[df['genre']=='direct'].iloc[:,4:].sum()/df.shape[0]).sort_values(ascending=False).index

Index(['aid_related', 'request', 'direct_report', 'food', 'other_aid',
       'weather_related', 'shelter', 'water', 'earthquake', 'medical_help',
       'medical_products', 'buildings', 'infrastructure_related', 'storm',
       'floods', 'death', 'clothing', 'search_and_rescue', 'transport',
       'other_weather', 'other_infrastructure', 'refugees', 'money',
       'security', 'missing_people', 'electricity', 'aid_centers', 'cold',
       'hospitals', 'military', 'offer', 'fire', 'shops', 'tools',
       'child_alone'],
      dtype='object')